MAP@K permits the evaluation of multiple predictions for a single row. It's often used for reccomendation systems

more resoure : [Asher U.](https://medium.com/kbtg-life/%E0%B8%97%E0%B8%B3%E0%B9%84%E0%B8%A1-mean-average-precision-at-k-map-k-%E0%B8%96%E0%B8%B6%E0%B8%87%E0%B8%A1%E0%B8%B5%E0%B8%9B%E0%B8%A3%E0%B8%B0%E0%B9%82%E0%B8%A2%E0%B8%8A%E0%B8%99%E0%B9%8C%E0%B8%81%E0%B8%B1%E0%B8%9A%E0%B9%82%E0%B8%88%E0%B8%97%E0%B8%A2%E0%B9%8C-classification-model-c9ad47d51608)

In [162]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
# ref: https://www.kaggle.com/code/wlifferth/generating-map-k-predictions/notebook

In [163]:
train_df = pd.read_csv('./datasets/BorneDiseaseDataset/train.csv')
train_df.head()

,id,sudden_fever,headache,mouth_bleed,nose_bleed,muscle_pain,joint_pain,vomiting,rash,diarrhea,...,breathing_restriction,toe_inflammation,finger_inflammation,lips_irritation,itchiness,ulcers,toenail_loss,speech_problem,bullseye_rash,prognosis
0,0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Lyme_disease
1,1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Tungiasis
2,2,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,Lyme_disease
3,3,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Zika
4,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,Rift_Valley_fever


In [164]:
train_df['prognosis'].value_counts()

prognosis
West_Nile_fever          85
Japanese_encephalitis    81
Tungiasis                70
Rift_Valley_fever        70
Chikungunya              66
Dengue                   63
Yellow_Fever             61
Zika                     58
Plague                   53
Lyme_disease             52
Malaria                  48
Name: count, dtype: int64

In [165]:
encoder = OrdinalEncoder()
train_df['prognosis'] = encoder.fit_transform(train_df[['prognosis']])
train_df['prognosis'][:10]

0     3.0
1     7.0
2     3.0
3    10.0
4     6.0
5     3.0
6     8.0
7     7.0
8     4.0
9     0.0
Name: prognosis, dtype: float64

In [166]:
encoder.categories_

[array(['Chikungunya', 'Dengue', 'Japanese_encephalitis', 'Lyme_disease',
        'Malaria', 'Plague', 'Rift_Valley_fever', 'Tungiasis',
        'West_Nile_fever', 'Yellow_Fever', 'Zika'], dtype=object)]

In [167]:
X_train, X_test, y_train, y_test = train_test_split(train_df.drop('prognosis', axis=1), train_df['prognosis'], random_state=31, test_size=0.1)

In [168]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((636, 65), (71, 65), (636,), (71,))

In [169]:
# training and predict probabiliities
lr_clf = LogisticRegression(solver='lbfgs', max_iter=4000)
lr_clf.fit(X_train, y_train)
predictions = lr_clf.predict_proba(X_test)

In [170]:
predictions[0], predictions.shape

(array([9.82485180e-03, 3.68607637e-01, 5.27947658e-02, 1.95002000e-05,
        4.04399863e-04, 6.30745545e-04, 1.15904990e-01, 3.05896918e-01,
        1.29613830e-01, 9.02574335e-03, 7.27661856e-03]),
 (71, 11))

<img src="https://www.sharpsightlabs.com/wp-content/uploads/2022/04/numpy-argsort_simple-visual-example.png" width=400px>

In [171]:
# take a look at getting our top 3 prognoses
sorted_prediction_ids = np.argsort(-predictions[0]) # reverse argsort
print("Output of predict_proba:")
print(predictions[0])
print(sorted_prediction_ids)

Output of predict_proba:
[9.82485180e-03 3.68607637e-01 5.27947658e-02 1.95002000e-05
 4.04399863e-04 6.30745545e-04 1.15904990e-01 3.05896918e-01
 1.29613830e-01 9.02574335e-03 7.27661856e-03]
[ 1  7  8  6  2  0  9 10  5  4  3]


In [172]:
predictions[0][1] == np.max(predictions[0]) # yes, index 1 actually the most value

True

In [173]:
top_3_prediction_ids = sorted_prediction_ids[:3]
top_3_predictions = encoder.inverse_transform(top_3_prediction_ids.reshape(-1,1))
print(top_3_prediction_ids)
print(top_3_predictions)

[1 7 8]
[['Dengue']
 ['Tungiasis']
 ['West_Nile_fever']]


In [174]:
# sort axis=1 by/each row
sorted_prediction_ids = np.argsort(-predictions, axis=1)
top_3_prediction_ids = sorted_prediction_ids[:, :3]
top_3_prediction_ids[:5], top_3_prediction_ids.shape

(array([[ 1,  7,  8],
        [ 4,  2, 10],
        [ 1,  7,  0],
        [ 1,  8,  7],
        [ 7,  6, 10]], dtype=int64),
 (71, 3))

In [175]:
original_shape = top_3_prediction_ids.shape
top_3_predictions = encoder.inverse_transform(top_3_prediction_ids.reshape(-1,1))
top_3_predictions = top_3_predictions.reshape(original_shape)
top_3_predictions[:5]

array([['Dengue', 'Tungiasis', 'West_Nile_fever'],
       ['Malaria', 'Japanese_encephalitis', 'Zika'],
       ['Dengue', 'Tungiasis', 'Chikungunya'],
       ['Dengue', 'West_Nile_fever', 'Tungiasis'],
       ['Tungiasis', 'Rift_Valley_fever', 'Zika']], dtype=object)

**Calculating MAP@K on our validation set**
- we have our top K (3) predictions

In [176]:
def apk(actual, predicted, k=3) :
    """
    compute the average precision at k
    """
    if len(predicted) > k :
        predicted = predicted[:k]
    
    score = 0.0
    num_hits = 0.0
    
    for i, p in enumerate(predicted) :
        if (p in actual) and p not in predicted[:i] :
            num_hits += 1.0
            score += num_hits/(i+1.0)
    
    if not actual :
        return 0.0
    
    return score / min(len(actual), k)

def mapk(actual, predicted, k=3) :
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [177]:
mapk(y_test.values.reshape(-1,1), top_3_prediction_ids, k=3)

0.2652582159624413

In [178]:
# generating test prediction
test_df = pd.read_csv('./datasets/BorneDiseaseDataset/test.csv')
test_df.head()

,id,sudden_fever,headache,mouth_bleed,nose_bleed,muscle_pain,joint_pain,vomiting,rash,diarrhea,...,lymph_swells,breathing_restriction,toe_inflammation,finger_inflammation,lips_irritation,itchiness,ulcers,toenail_loss,speech_problem,bullseye_rash
0,707,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,708,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,709,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,710,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,711,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [179]:
predictions = lr_clf.predict_proba(test_df)
sorted_prediction_ids = np.argsort(-predictions, axis=1)
top_3_prediction_ids = sorted_prediction_ids[:, :3]

original_shape = top_3_prediction_ids.shape
top_3_predictions = encoder.inverse_transform(top_3_prediction_ids.reshape(-1,1))
top_3_predictions = top_3_predictions.reshape(original_shape)
top_3_predictions[:5]

array([['Tungiasis', 'Rift_Valley_fever', 'Zika'],
       ['Dengue', 'Chikungunya', 'Malaria'],
       ['Japanese_encephalitis', 'West_Nile_fever', 'Yellow_Fever'],
       ['Japanese_encephalitis', 'Rift_Valley_fever', 'Tungiasis'],
       ['West_Nile_fever', 'Plague', 'Malaria']], dtype=object)

In [180]:
np.apply_along_axis(lambda x: np.array(' '.join(x), dtype='object'), axis=1, arr=top_3_predictions)[:5]

array(['Tungiasis Rift_Valley_fever Zika', 'Dengue Chikungunya Malaria',
       'Japanese_encephalitis West_Nile_fever Yellow_Fever',
       'Japanese_encephalitis Rift_Valley_fever Tungiasis',
       'West_Nile_fever Plague Malaria'], dtype=object)

In [181]:
# add column prediction: prognosis
test_df['prognosis'] = np.apply_along_axis(lambda x: np.array(' '.join(x), dtype='object'), axis=1, arr=top_3_predictions)
test_df['prognosis'][:10]

0                     Tungiasis Rift_Valley_fever Zika
1                           Dengue Chikungunya Malaria
2    Japanese_encephalitis West_Nile_fever Yellow_F...
3    Japanese_encephalitis Rift_Valley_fever Tungiasis
4                       West_Nile_fever Plague Malaria
5                             Zika Yellow_Fever Plague
6                 Malaria Japanese_encephalitis Plague
7                  Dengue Yellow_Fever West_Nile_fever
8            Yellow_Fever Plague Japanese_encephalitis
9                          Plague Yellow_Fever Malaria
Name: prognosis, dtype: object

In [183]:
test_df.to_csv('./datasets/BorneDiseaseDataset/submission.csv', columns=['id', 'prognosis'], index=False)